In [1]:
import pandas as pd

df_vaf = pd.read_csv ("/data/project/Meningioma/51.Amplicon/02.multiplex/07.pysam/Dura_vaf_df.tsv", sep = "\t", index_col = 0)
df_vaf = round ( df_vaf * 100 , 2)
#df_vaf.to_csv ("/data/project/Meningioma/51.Amplicon/02.multiplex/07.pysam/Dura_vaf_df.tsv", sep = "\t", index = True, header = True)
df_vaf

/opt/Yonsei/python/3.8.1/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.2' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


,chr22:29636758(NF2),chr22:29604113(NF2),chr22:29604050(NF2),chr22:29639185(NF2),chr16:2175609(TRAF7),chr14:104780214(AKT1),chr22:29604167(NF2),chr9:107487067(KLF4),chr22:29661335(NF2),chr16:2176145(TRAF7),chr22:29674837(NF2),chr22:29673365(NF2),chr22:29639168(NF2)
190426_Multiplex.sorted,0.28,0.00,0.0,0.0,0.13,0.11,0.0,0.28,0.00,0.0,0.0,0.05,0.00
221202_Multiplex.sorted,0.03,0.00,0.0,0.0,0.29,0.19,0.0,0.07,0.00,0.0,0.0,0.06,0.00
230127_Multiplex.sorted,0.02,0.00,0.0,0.0,0.09,0.03,0.0,0.09,0.00,0.0,0.0,0.06,0.00
230303_Multiplex.sorted,0.03,0.00,0.0,0.0,0.10,0.03,0.0,0.17,0.00,0.0,0.0,0.06,0.00
230323_Multiplex.sorted,0.03,0.00,0.0,0.0,0.18,0.04,0.0,0.26,1.43,0.0,0.0,0.10,0.00
230405_Multiplex.sorted,0.04,0.00,0.0,0.0,0.15,0.07,0.0,0.39,0.00,0.0,0.0,0.08,0.01
230822_Multiplex.sorted,0.02,0.00,0.0,0.0,0.17,0.02,0.0,0.29,0.00,0.0,0.0,0.13,0.00
230920_Multiplex.sorted,0.03,0.01,0.0,0.0,0.12,0.01,0.0,0.38,0.00,0.0,0.0,0.08,0.07


In [ ]:
import pysam
import numpy as np

class Variant:
    def __init__ (self, date, bamtype, BAM_Dir, chr_, pos_):
        self.date = date
        self.bamtype = bamtype
        self.BAM_Dir =  BAM_Dir
        self.chr_ = chr_
        self.pos_ = pos_
    def pysam_read (self):
        samfile = pysam.AlignmentFile(self.BAM_Dir, 'rb')
        #for ID in _variant_dic:
        multiallele_check = set()
        del_count, del_list = 0, []
        base_counts = {"A": 0, "T": 0, "C": 0, "G": 0, "N" : 0, "Del": 0}

        iter = samfile.fetch(contig = self.chr_, start = self.pos_-1, end = self.pos_)
        for i, read in enumerate(iter):
            if read.is_proper_pair:
                if read.flag > 512:
                    continue
                read_info = str(read).rstrip().split('\t')
                cigar_string = read.cigarstring
                start_site = read.reference_start + 1
                end_site = read.reference_end
                paired_read_start_site = read.next_reference_start + 1
                read_base = read.query_sequence
                read_length = read.query_alignment_length
                soft_al = read.query_alignment_start ### without clipped base
                soft_al2 = read.query_alignment_end ### without clipped base
                # if ('H' in cigar_string) | ('S' in cigar_string):
                #     continue
            
                if 'D' in cigar_string:
                    pp = start_site
                    for j in cigar_string.split("D")[0].split("M")[0:-1]:
                        if ("I" in j) | ("S" in j):
                            continue
                        else:
                            pp += int (j)
                    if pp - 1== int(self.pos_):
                        base_counts["Del"] += 1
                        #del_list.append (read.query_name)
                        del_list.append (read.cigarstring)
                else:
                    base_counts [ read.query_alignment_sequence [ self.pos_ - start_site]] += 1

        print ( "{}_Dura-{}".format (self.date, self.bamtype))
        print ("\t# of total read depth : {}". format ( np.sum ( list ( base_counts.values() )  ) ) )
        for key in base_counts.keys():
            print ("\t# of {} : {}". format (key, base_counts[key]))
        print ("\t\tdel_list = {}".format (del_list))


#BAM_Dir, chr_, pos_ = "/data/project/Meningioma/02.Align/hg38/Dura/05.Final_bam/230419_Dura_no_softclips.bam", "chr22", 29674838                  # WES




# Dura_230323_WES1 = Variant ( 230323, "WES1", "/data/project/Meningioma/02.Align/hg38/Dura/05.Final_bam/230323_Dura.bam", "chr22", 29661335  )
# Dura_230323_WES2 = Variant ( 230323, "WES2", "/data/project/Meningioma/02.Align/hg38/Dura/05.Final_bam/230323_2_Dura.bam", "chr22", 29661335  )
# Dura_230323_WES1.pysam_read()
# Dura_230323_WES2.pysam_read()
        

# Dura_230405_WES1 = Variant ( 230405, "WES1", "/data/project/Meningioma/02.Align/hg38/Dura/05.Final_bam/230405_Dura.bam", "chr9", 107487067  )
# Dura_230405_WES2 = Variant ( 230405, "WES2", "/data/project/Meningioma/02.Align/hg38/Dura/05.Final_bam/230405_2_Dura.bam", "chr9", 107487067  )
# Dura_230405_duplicated = Variant ( 230405, "duplicated", "/data/project/Meningioma/00.Targetted/02.Align/hg38/Dura/01.Pre_bam/230405.sorted.bam", "chr9", 107487067  )
# Dura_230405_deduplicated = Variant ( 230405, "deduplicated", "/data/project/Meningioma/00.Targetted/02.Align/hg38/Dura/05.Final_bam/230405_Dura.bam", "chr9", 107487067  )
# Dura_230405_WES1.pysam_read()
# Dura_230405_WES2.pysam_read()
# Dura_230405_duplicated.pysam_read()
# Dura_230405_deduplicated.pysam_read()


Dura_230405_Multiplex_duplicated = Variant ( 230405, "duplicated", "/data/project/Meningioma/00.Targetted/02.Align/hg38/Dura/01.Pre_bam/230405_Multiplex.sorted.bam", "chr9", 107487067  )
Dura_230405_Multiplex_duplicated.pysam_read()
        
# Dura_230419_WES = Variant ( 230419, "WES", "/data/project/Meningioma/02.Align/hg38/Dura/05.Final_bam/230419_Dura.bam", "chr22", 29674837 )
# Dura_230419_duplicated = Variant ( 230419, "duplicated", "/data/project/Meningioma/00.Targetted/02.Align/hg38/Dura/01.Pre_bam/230419.sorted.bam", "chr22", 29674837 )
#Dura_230419_deduplicated = Variant ( 230419, "deduplicated", "/data/project/Meningioma/00.Targetted/02.Align/hg38/Dura/06.Final_bam/230419_Dura.bam", "chr22", 29674837 )
# Dura_230419_WES.pysam_read()
# Dura_230419_duplicated.pysam_read()
#Dura_230419_deduplicated.pysam_read()
# Dura_230419_duplicated = Variant ( 230419, "duplicated", "/data/project/Meningioma/00.Targetted/02.Align/hg38/Dura/01.Pre_bam/230419.sorted.bam", "chr22", 29674836 )
# Dura_230419_duplicated.pysam_read()
# Dura_230419_duplicated = Variant ( 230419, "duplicated", "/data/project/Meningioma/00.Targetted/02.Align/hg38/Dura/01.Pre_bam/230419.sorted.bam", "chr22", 29674838 )
# Dura_230419_duplicated.pysam_read()

# Dura_230822_WES = Variant ( 230822, "WES", "/data/project/Meningioma/02.Align/hg38/Dura/05.Final_bam/230822_Dura.bam", "chr22", 29673365  )
# Dura_230822_duplicated = Variant ( 230822, "duplicated", "/data/project/Meningioma/00.Targetted/02.Align/hg38/Dura/01.Pre_bam/230822.sorted.bam", "chr22", 29673365  )
# Dura_230822_deduplicated = Variant ( 230822, "deduplicated", "/data/project/Meningioma/00.Targetted/02.Align/hg38/Dura/05.Final_bam/230822_Dura.bam", "chr22", 29673365  )
# Dura_230822_WES.pysam_read()
# Dura_230822_duplicated.pysam_read()
# Dura_230822_deduplicated.pysam_read()

230419_Dura-WES
	# of total read depth : 317
	# of A : 0
	# of T : 0
	# of C : 0
	# of G : 315
	# of N : 0
	# of Del : 2
		del_list = ['127M1D24M', '95M1D56M']
230419_Dura-duplicated
	# of total read depth : 9017309
	# of A : 7557
	# of T : 4428
	# of C : 966
	# of G : 8997491
	# of N : 0
	# of Del : 6867
		del_list = ['124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D23M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D27M', '124M1D20M', '124M1D27M', '124M1D27M', '12

In [14]:
import pysam

# File path
BAM_Dir, chr_, pos_ = "/data/project/Meningioma/02.Align/hg38/Dura/05.Final_bam/230822_Dura.bam", "chr22", 29673365
BAM_Dir, chr_, pos_ = "/data/project/Meningioma/00.Targetted/02.Align/hg38/Dura/05.Final_bam/230419_Dura.bam", "chr22", 29674836


# Load the bam file
samfile = pysam.AlignmentFile(BAM_Dir, "rb")


# Dictionary to store counts for each base
base_counts = {"A": 0, "T": 0, "C": 0, "G": 0, "N": 0, "del": 0}

# Iterate over the reads covering the position
t = 0
for pileupcolumn in samfile.pileup(chr_, pos_, pos_ + 1):
    t += 1
    if pileupcolumn.pos == pos_:
        for pileupread in pileupcolumn.pileups:
            if pileupread.is_del:
                base_counts["del"] += 1
                print (pileupread)
            #elif not pileupread.is_refskip:  # query pos_ is None if is_del or is_refskip is set
            else:
                base = pileupread.alignment.query_sequence[pileupread.query_position]
                if base in base_counts:
                    base_counts[base] += 1
                else:
                    base_counts[base] = 1

for base, count in base_counts.items():
    print(f"The count of '{base}' at position {chr_}:{pos_ + 1} is: {count}")

# Close the bam file
samfile.close()

The count of 'A' at position chr22:29674837 is: 0
The count of 'T' at position chr22:29674837 is: 0
The count of 'C' at position chr22:29674837 is: 0
The count of 'G' at position chr22:29674837 is: 9
The count of 'N' at position chr22:29674837 is: 0
The count of 'del' at position chr22:29674837 is: 0
